<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_NSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gluonnlp tqdm

     |████████████████████████████████| 344 kB 5.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595730 sha256=b1eccc72399407343aedb900c7d16401450def3b9f04f07856495306a20e6165
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [2]:
pip install mxnet

     |████████████████████████████████| 47.3 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
pip install transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 6.5 MB 45.4 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Softmax
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [5]:
# 모델과 tokenizer 준비

model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

# 동일 문장 여부 파악

In [6]:
# 임의의 문장 예측 (네이버 기사)

# 이어지는 문장
prompt1 = '정부는 약사 등의 '
next_sentence1 = '환자 발생으로 약국 영업이 중단될 경우 해당 지역의 보건 서비스에 심각한 영향이 우려돼 이번 지침을 마련했다고 설명했다.'

# 이어지지 않는 문장
prompt2 = '튜브 겉면에도 복합 재질을 의미하는 '
next_sentence2 = '연기를 할 수 없어 이같은 방법을 선택한 것으로 보인다.'

In [7]:
def same_sentence(prompt, next):
  encoding = tokenizer(prompt, next, return_tensors='tf')
  logits = model(encoding['input_ids'], token_type_ids = encoding['token_type_ids'])[0]
  softmax = Softmax()
  probs = softmax(logits)
  res = tf.math.argmax(probs, axis=-1).numpy()

  return res

In [8]:
res1 = same_sentence(prompt1, next_sentence1)

if res1 == 0:
  print('두 문장은 이어지는 문장입니다.')
else:
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지는 문장입니다.


In [9]:
res2 = same_sentence(prompt2, next_sentence2)

if res2 == 0:
  print('두 문장은 이어지는 문장입니다.')
else: 
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지지 않는 문장입니다.


# 데이터셋 준비

In [18]:
import os

root = '/content/drive/MyDrive/news_class9x13000/'
categories = os.listdir(root)

dataset = []

for category in categories:
  files = os.listdir(root + category)
  for i, f in enumerate(files):
    fname = root + category + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read() + '\n'
    dataset.append([strings])
    file.close()
  

In [19]:
dataset[1]

['돌봐주는 대상과 손봐주는 대상 정해져있다?\n친이무죄 반이유죄.\n이 같은 말풍선이 교육계 안팎에서 터져 나오고 있다.\n이 대통령과 친한 단체(사람)가 하면 무죄이고, 이 대통령을 반대하는 단체(사람)가 하면 유죄라는 얘기다.\n정치후원금 사태, 한쪽은 된서리 다른 쪽은?\n사건의 발단은 교사시국선언과 정당 정치 후원금 문제에서 시작했다.\n이 대통령 반대 편에 선 전교조. 시국선언과 일부 조합원의 민주노동당에 대한 정치 후원금 기부 등으로 된서리를 맞고 있다.\n전교조 본부 사무실 압수수색 등과 통장, 전화 통화, 인터넷 메일 내역 등에 대한 이 잡기식 수사가 아직도 진행 중이다.\n대선 시절부터 친이 단체란 지적을 받은 한국교총. 이 단체 또한 교사 시국선언을 했고, 일부 소속원이 한나라당 후원금을 낸 사실이 들통 났는데도 아직까지 특별한 검경의 움직임이 없는 것처럼 보인다.\n어제오늘 논란이 된 전교조의 서명운동과 교사결의대회 계획에 대한 교과부의 호들갑은 더 심각하다.\n전교조가 5월부터 10만 교사 서명운동과 5월 중 결의대회를 벌이기로 한 것에 대해 교과부는 징계 으름장을 놓았다.\n<한겨레> 1일치 보도를 보면 이성희 교과부 학교자율화추진관은 공무원 복무규정을 보면, 공무원은 정부 정책에 대한 비판을 하지 못하도록 돼 있다.\n이를 어기면 징계할 수밖에 없다고 말했다고 한다.\n군부독재 시절에도 벌인 서명운동과 결의대회에 대해서도 족쇄를 채우겠다는 얘기로 해석된다.\n교총이 하면 돌봐주고 전교조가 하면 손봐주고\n하지만 한국교총은 이명박 정부가 들어선 뒤인 지난 2008년 10월 정부 교육정책에 반대하는 교육세 폐지 반대 서명운동을 벌였다.\n같은 해 11월 25일 교총은 교원 22만 명의 서명을 받았다면서 기자회견까지 열었다.\n당시 교과부는 눈을 질끈 감아줬다.\n교총이 하면 돌봐주고, 전교조가 하면 손봐준다는 지적을 피할 수 없는 대목이다.\n물론 교과부가 얘기한 대통령령인 공무원 복무규정에는 집단으로 또는 단체의 명의를 사용하여 국가의

In [20]:
table = str.maketrans({
    '\'': '',
    '\"': '',
    '★': '',
    '▲': '',
    '▶': '',
    '┌': '',
    '├': '',
    '│': '',
    '└': '',
    '→': '',
    '‘': '',
    '■': '',
    '○': ''

})

In [ ]:
process_dataset = []

for sentences in dataset:
  sentences = sentences[0]
  sentences = sentences.translate(table)
  sentences = sentences.split('\n')
  del sentences[0]
  process_dataset.extend(sentences)

process_dataset

In [22]:
len(process_dataset)

4200445

In [23]:
with open(
    os.path.join(root, 'process data'), mode = 'w', encoding = 'utf-8'
) as f:
  for data in process_dataset:
    f.write(data)

print('save the preprocessing data (del title version) file')

save the preprocessing data (del title version) file


In [35]:
train_dataset = process_dataset[:15000]
test_dataset = process_dataset[15000:25000]

len(train_dataset), len(test_dataset)

(15000, 10000)

In [36]:
train_dataset[14999]

'라고 말한다.'

In [37]:
test_dataset[9999]

'이어 그는 한태식 위원은 인권위가 야간집회에 대해 헌법재판소에 의견을 내는 것은 사법부의 독립성을 해친다고 말한 위원이라며 사법부, 입법부, 행정부를 향해 인권적인 결정을 내리도록 의견을 내는 것이 인권위의 역할인데, 이 조차 모르면서 인권위의 파행을 운운하고 있다고 비판했다.'

In [38]:
def save_file(dataset, fname):
  with open(
      os.path.join(root, fname), mode = 'w', encoding = 'utf-8'
  ) as f:
    for data in dataset:
      f.write(data)
  
save_file(train_dataset, 'train')
save_file(test_dataset, 'test')

## 데이터 만들기
# 필요 없는 부분 

    corpus = [prompt, next, isNext]

    for index in range(process_dataset):
      if len(corpus) % 2 == 0: 
        이어지는 문장
        corpus.append([process_dataset[index - 1], process_dataset[index], True])
      else: 
        이어지지 않는 문장
        randomIndex = randint(0, len(corpus)-1)
        corpus.append([process_dataset[randomIndex], process_dataset[index], False])





In [ ]:
from random import randint

corpus = []

for index in range(0, len(process_dataset)-1):
  # 기사의 마지막 부분은 패스
  if process_dataset[index] == 'end':
    pass
  if process_dataset[index + 1] == 'end':
    pass

  if len(corpus) % 2 == 0:
    # 이어지는 문장
    prompt = process_dataset[index]
    next = process_dataset[index + 1]
    if same_sentence(prompt, next) == 0:
      corpus.append([prompt, next, True])
    else: 
      corpus.append([prompt, next, False])
  else:
    # 이어지지 않는 문장
    random_index = randint(0, len(process_dataset)-1)
    corpus.append([process_dataset[index], process_dataset[random_index], False])

len(corpus)

In [33]:
len(corpus)

77968

In [ ]:
import pandas as pd

corpus_dataset = pd.DataFrame(corpus, columns = ['prompt', 'next', 'isNext'])

corpus_dataset

In [35]:
corpus_dataset.isNext.value_counts()

False    44257
True     33711
Name: isNext, dtype: int64

In [37]:
type(corpus_dataset)

pandas.core.frame.DataFrame

In [39]:
corpus_dataset.prompt[3]

'올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.'

In [40]:
from sklearn.model_selection import train_test_split

target = 'isNext'

train, test = train_test_split(corpus, test_size = 0.2, random_state = 42)
len(train), len(test)

(62374, 15594)

In [42]:
train_data = pd.DataFrame(train, columns = ['prompt', 'next', 'isNext'])
train_data.isNext.value_counts()

False    35450
True     26924
Name: isNext, dtype: int64

In [43]:
test_data = pd.DataFrame(test, columns = ['prompt', 'next', 'isNext'])
test_data.isNext.value_counts()

False    8807
True     6787
Name: isNext, dtype: int64

In [44]:
import csv

def save_file(fname, dataset):
  with open(os.path.join(root, fname), 'w', encoding='utf-8', newline='') as f:
    tw = csv.writer(f, delimiter='\t')
    for data in corpus:
      tw.writerow(data)

save_file('train.tsv', train)
save_file('test.tsv', test)

In [19]:
corpus_dataset.isNext.value_counts()

False    5675
True     4324
Name: isNext, dtype: int64

# Fine tuning

1. 데이터셋준비 (완료)
2. 데이터 정제
3. 데이터 토큰화
4. 데이터 패딩 및 attention mask
5. train, val, test dataset
6. train, val, test를 torch tensor로 변환
7. TensorDataset으로 train input, mask, label 묶기
8. 모델 device를 `cuda:0`으로 설정
 (`model.to(device)` )
9. BertForSequenceClassification.from_pretrained => pretrained 된 bert 불러오기

  [간단한 예제](https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/)

10. 옵티마이저 설정
11. 스케줄러 설정
12. 랜덤 시드 고정
13. 모델 그래디언트 초기화
14. Epoch만큼 훈련 반복

참고 : [BERT 모델의 장점과 Fine-Tuning 흐름 정리](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=antler07&logNo=222066071573)

[참고 링크 - 블로그](https://velog.io/@jiyoung/Text-ClassificationKoBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0-%EC%BD%94%EB%93%9C)

[참고 링크 - 노션](https://www.ohsuz.dev/4bb36581-b0bd-49ca-acd0-530c35546009)

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [28]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [29]:
# GPU setting
device = torch.device('cuda:0')

In [30]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [45]:
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional
from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer

import logging


logger = logging.getLogger(__name__)

DEPRECATION_WARNING = (
    "This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets "
    "library. You can have a look at this example script for pointers: {0}"
)

class TextDatasetForNextSentencePrediction(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: tokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        short_seq_probability=0.1,
        nsp_probability=0.5,
    ):
        warnings.warn(
            DEPRECATION_WARNING.format(
                "https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py"
            ),
            FutureWarning,
        )
        if not os.path.isfile(file_path):
            raise ValueError(f"Input file path {file_path} not found")

        self.short_seq_probability = short_seq_probability
        self.nsp_probability = nsp_probability

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory,
            f"cached_nsp_{tokenizer.__class__.__name__}_{block_size}_{filename}",
        )

        self.tokenizer = tokenizer

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"

        # Input file format:
        # (1) One sentence per line. These should ideally be actual sentences, not
        # entire paragraphs or arbitrary spans of text. (Because we use the
        # sentence boundaries for the "next sentence prediction" task).
        # (2) Blank lines between documents. Document boundaries are needed so
        # that the "next sentence prediction" task doesn't span between documents.
        #
        # Example:
        # I am very happy.
        # Here is the second sentence.
        #
        # A new document.

        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.documents = [[]]
                with open(file_path, encoding="utf-8") as f:
                    while True:
                        line = f.readline()
                        if not line:
                            break
                        line = line.strip()

                        # Empty lines are used as document delimiters
                        if not line and len(self.documents[-1]) != 0:
                            self.documents.append([])
                        tokens = tokenizer.tokenize(line)
                        tokens = tokenizer.convert_tokens_to_ids(tokens)
                        if tokens:
                            self.documents[-1].append(tokens)

                logger.info(f"Creating examples from {len(self.documents)} documents.")
                self.examples = []
                for doc_index, document in enumerate(self.documents):
                    self.create_examples_from_document(document, doc_index, block_size)

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    f"Saving features into cached file {cached_features_file} [took {time.time() - start:.3f} s]"
                )

    def create_examples_from_document(self, document: List[List[int]], doc_index: int, block_size: int):
        """Creates examples for a single document."""

        max_num_tokens = block_size - self.tokenizer.num_special_tokens_to_add(pair=True)

        # We *usually* want to fill up the entire sequence since we are padding
        # to `block_size` anyways, so short sequences are generally wasted
        # computation. However, we *sometimes*
        # (i.e., short_seq_prob == 0.1 == 10% of the time) want to use shorter
        # sequences to minimize the mismatch between pretraining and fine-tuning.
        # The `target_seq_length` is just a rough target however, whereas
        # `block_size` is a hard limit.
        target_seq_length = max_num_tokens
        if random.random() < self.short_seq_probability:
            target_seq_length = random.randint(2, max_num_tokens)

        current_chunk = []  # a buffer stored current working segments
        current_length = 0
        i = 0

        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)
            if i == len(document) - 1 or current_length >= target_seq_length:
                if current_chunk:
                    # `a_end` is how many segments from `current_chunk` go into the `A`
                    # (first) sentence.
                    a_end = 1
                    if len(current_chunk) >= 2:
                        a_end = random.randint(1, len(current_chunk) - 1)

                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])

                    tokens_b = []

                    if len(current_chunk) == 1 or random.random() < self.nsp_probability:
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        # This should rarely go for more than one iteration for large
                        # corpora. However, just to be careful, we try to make sure that
                        # the random document is not the same as the document
                        # we're processing.
                        for _ in range(10):
                            random_document_index = random.randint(0, len(self.documents) - 1)
                            if random_document_index != doc_index:
                                break

                        random_document = self.documents[random_document_index]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        # We didn't actually use these segments so we "put them back" so
                        # they don't go to waste.
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # Actual next
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    if not (len(tokens_a) >= 1):
                        raise ValueError(f"Length of sequence a is {len(tokens_a)} which must be no less than 1")
                    if not (len(tokens_b) >= 1):
                        raise ValueError(f"Length of sequence b is {len(tokens_b)} which must be no less than 1")

                    # add special tokens
                    input_ids = self.tokenizer.build_inputs_with_special_tokens(tokens_a, tokens_b)
                    # add token type ids, 0 for sentence a, 1 for sentence b
                    token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(tokens_a, tokens_b)

                    example = {
                        "input_ids": torch.tensor(input_ids, dtype=torch.long),
                        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                        "next_sentence_label": torch.tensor(1 if is_random_next else 0, dtype=torch.long),
                    }

                    self.examples.append(example)

                current_chunk = []
                current_length = 0

            i += 1

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [61]:
# 파라미터 세팅

max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [49]:
# 토큰화
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [51]:
train = TextDatasetForNextSentencePrediction(
    tokenizer = tokenizer,
    file_path = '/content/drive/MyDrive/news_class9x13000/train',
    overwrite_cache = False,
    block_size = 128,
    short_seq_probability = 0.1,
    nsp_probability = 0.5,
)

test = TextDatasetForNextSentencePrediction(
    tokenizer = tokenizer,
    file_path = '/content/drive/MyDrive/news_class9x13000/test',
    overwrite_cache = False,
    block_size = 128,
    short_seq_probability = 0.1,
    nsp_probability = 0.5,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py


In [52]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train, batch_size=batch_size, num_workers=2)
test_dataloader = DataLoader(test, batch_size=batch_size, num_workers=2)

In [53]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [54]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [55]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [56]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [57]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [58]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [59]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [60]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: ignored

In [63]:
x = 0
for i in tqdm(enumerate(train_dataloader)):
  print(i)

0it [00:00, ?it/s]

(0, {'input_ids': tensor([[   2, 3701, 2292,  ..., 4538,   18,    3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1]]), 'next_sentence_label': tensor([1])})
